In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
df = pd.read_csv('./combined_data_notifications.csv')

# Siapkan model ANOVA dua arah
model = ols('Latency ~ C(Broker) + C(Requests) + C(Broker):C(Requests)', data=df).fit()

# Hitung tabel ANOVA
anova_table = sm.stats.anova_lm(model, typ=2)

# Tampilkan tabel ANOVA
print(anova_table)

                             sum_sq          df             F  PR(>F)
C(Broker)              3.748083e+17         1.0  1.278873e+05     0.0
C(Requests)            3.977622e+19         7.0  1.938848e+06     0.0
C(Broker):C(Requests)  8.576254e+17         7.0  4.180401e+04     0.0
Residual               5.640911e+19  19247201.0           NaN     NaN


In [5]:
print(df.groupby('Broker')['Latency'].describe())
print('-----------------------END OF LINE')
print(df.groupby('Requests')['Latency'].describe())
print('-----------------------END OF LINE')
print(df.describe())

            count          mean           std  min        25%        50%  \
Broker                                                                     
Kafka   9637981.0  3.024825e+06  2.296659e+06  3.0  1066509.0  2304488.0   
NATS    9609236.0  3.307741e+06  2.192831e+06  1.0  1493109.0  3243186.0   

               75%        max  
Broker                         
Kafka   5013261.00  7625819.0  
NATS    5019967.25  7625819.0  
-----------------------END OF LINE
               count          mean           std  min         25%        50%  \
Requests                                                                       
3             4774.0  1.575462e+03  1.557869e+03  1.0      238.25      749.0   
6            10723.0  4.201074e+03  2.646969e+03  2.0     1690.00     4317.0   
30           57277.0  2.612294e+04  1.494082e+04  2.0    14495.00    25677.0   
60          113553.0  3.542351e+04  3.843439e+04  2.0     4248.00    13660.0   
300         575336.0  2.761664e+05  1.489500e+05  4.

In [7]:
from scipy.stats import shapiro

shapiro_test = shapiro(df['Latency'])
print('Shapiro-Wilk test:', shapiro_test)

Shapiro-Wilk test: ShapiroResult(statistic=np.float64(0.9429475444401435), pvalue=np.float64(2.365092960700824e-140))


D:\..SKRIPSI\CALCULATION\e2e-latency\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 19247217.
  res = hypotest_fun_out(*samples, **kwds)


In [17]:
from scipy.stats import boxcox, shapiro
import numpy as np
    
df = pd.read_csv('./combined_data_notifications.csv')

# Terapkan transformasi Box-Cox
df['Latency_boxcox'] = np.sqrt(df['Latency'])
df['Latency_sqrt'], _ = boxcox(df['Latency'] + 1)  # Menambahkan 1 untuk menghindari masalah dengan nilai 0
df['Latency_log'] = np.log(df['Latency'] + 1)

# Lakukan uji Shapiro-Wilk lagi pada data yang telah ditransformasi
shapiro_test_boxcox = shapiro(df['Latency_boxcox'])
print('Shapiro-Wilk test (Box-Cox-transformed):', shapiro_test_boxcox)

shapiro_test_sqrt = shapiro(df['Latency_sqrt'])
print('Shapiro-Wilk test  (Box-Cox-transformed):', shapiro_test_sqrt)

shapiro_test_log = shapiro(df['Latency_log'])
print('Shapiro-Wilk test  (Box-Cox-transformed):', shapiro_test_log)
    
# # Siapkan model ANOVA dua arah
# model = ols('Latency_boxcox ~ C(Broker) + C(Requests) + C(Broker):C(Requests)', data=df).fit()
# 
# # Hitung tabel ANOVA
# anova_table = sm.stats.anova_lm(model, typ=2)
# 
# # Tampilkan tabel ANOVA
# print(anova_table)

D:\..SKRIPSI\CALCULATION\e2e-latency\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:573: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 19247217.
  res = hypotest_fun_out(*samples, **kwds)


Shapiro-Wilk test (Box-Cox-transformed): ShapiroResult(statistic=np.float64(0.9563163590733877), pvalue=np.float64(2.1189354787363743e-132))
Shapiro-Wilk test  (Box-Cox-transformed): ShapiroResult(statistic=np.float64(0.9578975618048526), pvalue=np.float64(2.545203109006421e-131))
Shapiro-Wilk test  (Box-Cox-transformed): ShapiroResult(statistic=np.float64(0.7442125074048933), pvalue=np.float64(2.0543042958480136e-189))


In [14]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

df = pd.read_csv('./combined_data_notifications.csv')

# Check initial data
print(df)

# Perform Box-Cox transformation
df['Latency_boxcox'], _ = boxcox(df['Latency'] + 1) # Adding 1 to avoid log(0)

# Verify the data is now clean
print(df.describe())

# Prepare the ANOVA model with Box-Cox transformed data
model = ols('Latency_boxcox ~ C(Broker) + C(Requests) + C(Broker):C(Requests)', data=df).fit()

# Perform ANOVA
anova_table = sm.stats.anova_lm(model, typ=2)

# Display ANOVA results
print(anova_table)

          Latency Broker  Requests
0              45  Kafka         3
1             148  Kafka         3
2             155  Kafka         3
3             162  Kafka         3
4             165  Kafka         3
...           ...    ...       ...
19247212  2807348   NATS      6000
19247213  2807266   NATS      6000
19247214  2807172   NATS      6000
19247215  2807098   NATS      6000
19247216  2807032   NATS      6000

[19247217 rows x 3 columns]


MemoryError: Unable to allocate 587. MiB for an array with shape (2, 19247217) and data type complex128